In [2]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from keras.preprocessing import text, sequence
from scipy import sparse
from tqdm import tqdm
import pickle
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [3]:
tt = pd.read_pickle(FILE.train_final.value)
te = pd.read_pickle(FILE.test_final.value)
tt.drop('time_hour',axis = 1,inplace=True)
te.drop('time_hour',axis = 1,inplace=True)
da = pd.concat([tt, te], axis=0, ignore_index=True,sort=False)

In [4]:
col = 'model'
da[col] = da[col].astype(str)
tt[col] = tt[col].astype(str)
te[col] = te[col].astype(str)
tok=text.Tokenizer(num_words=da[col].nunique(),lower=False)
tok.fit_on_texts(list(da[col]))
seq = tok.texts_to_sequences(list(da[col]))

csr_save = sparse.lil_matrix((len(da),len(tok.index_word)+1))
for row in tqdm(range(csr_save.shape[0])):
    for col in seq[row]:
        csr_save[row,col] += 1
        
csr_train = sparse.csr_matrix(csr_save[:len(tt),:])
csr_test = sparse.csr_matrix(csr_save[len(tt):,:])
print(csr_train.shape)
col = 'model'
pickle.dump(csr_train,open(FILE.one_hot_train_formatter.value.format(col),'wb'))
pickle.dump(csr_test,open(FILE.one_hot_test_formatter.value.format(col),'wb'))

100%|██████████| 3072915/3072915 [00:31<00:00, 98763.51it/s]


(2992639, 13611)


# V2 - original

In [27]:
tt = pd.read_pickle(FILE.train_final.value)
te = pd.read_pickle(FILE.test_final.value)
tt.drop('time_hour',axis = 1,inplace=True)
te.drop('time_hour',axis = 1,inplace=True)
da = pd.concat([tt, te], axis=0, ignore_index=True,sort=False)

enc = OneHotEncoder()
da = da.fillna(-1)
special = ['model']
for i in special:
    da[i] = da[i].map(dict(zip(da[i].unique(), range(0, da[i].nunique()))))
tt_x = da[special].iloc[:len(tt)].copy()
te_x = da[special].iloc[len(tt):].copy()
for feature in tqdm(special):

    print('build new')

    enc.fit(da[feature].values.reshape(-1, 1))
    onehot_train = enc.transform(tt_x[feature].values.reshape(-1, 1))
    onehot_test = enc.transform(te_x[feature].values.reshape(-1, 1))
    pickle.dump(onehot_train,open(FILE.one_hot_train_formatter.value.format(feature),'wb'))
    pickle.dump(onehot_test,open(FILE.one_hot_test_formatter.value.format(feature),'wb'))

  0%|          | 0/1 [00:00<?, ?it/s]

build new


100%|██████████| 1/1 [00:00<00:00,  3.02it/s]


# V3 - clean

In [22]:
tt = pd.read_pickle(FILE.train_final.value)
te = pd.read_pickle(FILE.test_final.value)
tt.drop('time_hour',axis = 1,inplace=True)
te.drop('time_hour',axis = 1,inplace=True)
da = pd.concat([tt, te], axis=0, ignore_index=True,sort=False)

In [23]:
import re
import pandas as pd
import urllib.parse
import numpy as np


pattern1=r'%2522|%(25){0,}2B|%20|%(25){0,}28|%(25){0,}29|%(25){0,}2C'
#用空格替代连字符"-或者+或者_"
pattern2=r'-{1,}|(?<=[a-zA-Z\d])\+{1,}\s{0,}(?=[a-zA-Z\d])|_{1,}'
#两个以上的空格都替换成一个
pattern3='\s{2,}'
#去掉括号
pattern4='\(|\)'

def reg(input_string):
    if isinstance(input_string, str):
        new_string=urllib.parse.unquote(input_string)
#         new_string=re.sub(pattern1,' ',new_string)
        new_string=re.sub(r' +',' ',new_string)
#         new_string = re.sub(pattern2,' ',new_string)
#         new_string = re.sub(pattern3,' ', new_string)
#         new_string = re.sub(pattern4,'', new_string)
        # 全部转为大写
        new_string = new_string.upper()
        if '%' in  new_string:
            pass
#             print(input_string,new_string)
#             new_string=np.nan
        return new_string
    else:
        # print(input_string)
        return input_string

da['model']=da['model'].astype(str).apply(reg)

In [24]:
enc = OneHotEncoder()
da = da.fillna(-1)
special = ['model']
for i in special:
    da[i] = da[i].map(dict(zip(da[i].unique(), range(0, da[i].nunique()))))
tt_x = da[special].iloc[:len(tt)].copy()
te_x = da[special].iloc[len(tt):].copy()
for feature in tqdm(special):

    print('build new')

    enc.fit(da[feature].values.reshape(-1, 1))
    onehot_train = enc.transform(tt_x[feature].values.reshape(-1, 1))
    onehot_test = enc.transform(te_x[feature].values.reshape(-1, 1))
    pickle.dump(onehot_train,open(FILE.one_hot_train_formatter.value.format(feature),'wb'))
    pickle.dump(onehot_test,open(FILE.one_hot_test_formatter.value.format(feature),'wb'))

  0%|          | 0/1 [00:00<?, ?it/s]

build new


100%|██████████| 1/1 [00:00<00:00,  3.25it/s]


# U_tag

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
tt = pd.read_pickle(FILE.train_final.value)
te = pd.read_pickle(FILE.test_final.value)
tt.drop('time_hour',axis = 1,inplace=True)
te.drop('time_hour',axis = 1,inplace=True)
da = pd.concat([tt, te], axis=0, ignore_index=True,sort=False)

cv = CountVectorizer(min_df=1)
    
for feature in ['user_tags']:

        da[feature] = da[feature].astype(str)
        cv.fit(da[feature])
        onehot_train = cv.transform(tt[feature].astype(str))
        onehot_test = cv.transform(te[feature].astype(str))
        pickle.dump(onehot_train,open(FILE.one_hot_train_formatter.value.format(feature),'wb'))
        pickle.dump(onehot_test,open(FILE.one_hot_test_formatter.value.format(feature),'wb'))